## Art Dating

#### Students
- Zhenbang Chen
- Zhenjia Chen

### Setup

Importing packages and dependencies.  Load dataset for categorization.

In [1]:
import os
import copy
from PIL import Image
rootpath = "."

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
# You might not have tqdm, which gives you nice progress bars
!pip install tqdm
from tqdm.auto import tqdm
import os
import copy
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print("Using the GPU!")
else:
    print("WARNING: Could not find GPU! Using CPU only")

PyTorch Version:  1.3.1
Torchvision Version:  0.4.2
Using the GPU!


### Model Initialization

In [3]:
def initialize_model(num_classes, resume_from=None):
    
    # Model (nn.Module) to return
    # model_ft = models.resnet18(pretrained = false)
    model_ft = models.resnet50(pretrained = true)
    
    in_features = 512
    model_ft.fc = nn.Linear(in_features, num_classes)
    

    return model_ft

### Tests

In [4]:
# Transform to apply to the data
# transform = torchvision.transforms.Compose([
#     torchvision.transforms.ToTensor(),
#     torchvision.transforms.Normalize(mean=(0.5,), std=(0.5,))
# ])

# Transform to apply to the data for use with pretrained ResNet model
transform = torchvision.transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [5]:
# Get training data from directory
art_train = torchvision.datasets.ImageFolder(root="./data/art_train",
                                                 transform=transform)

# Get testing data from directory
art_val = torchvision.datasets.ImageFolder(root="./data/art_val",
                                               transform=transform)

In [14]:
# Create random sampler
random_sampler = torch.utils.data.RandomSampler(data_source=art_train,
                                                replacement=True,
                                                num_samples=int(len(art_train)/40))

# Create batched dataloader
# art_train_loader = torch.utils.data.DataLoader(dataset=art_train,
#                                                    batch_size=8,
#                                                    shuffle=True,
#                                                    num_workers=4,
#                                                    pin_memory=True)

art_train_loader = torch.utils.data.DataLoader(dataset=art_train,
                                                   batch_size=8,
                                                   sampler=random_sampler,
                                                   shuffle=False,
                                                   num_workers=4,
                                                   pin_memory=True)

art_val_loader = torch.utils.data.DataLoader(dataset=art_val,
                                                 batch_size=8,
                                                 shuffle=False,
                                                 num_workers=4,
                                                 pin_memory=True)

### Initialization and Methods

In [8]:
# Initialize model
# model = torchvision.models.resnet18(pretrained=False)
model = torchvision.models.resnet50(pretrained=True)

# Set number of output classes
# model.conv1 = nn.Conv2d(in_channels=3,
#                         out_channels=64,
#                         kernel_size=(7,7),
#                         stride=(2,2),
#                         padding=(3,3),
#                         bias=False)

in_features = model.fc.in_features
out_features = 20
model.fc = nn.Linear(in_features, out_features)

model = model.to(device)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to C:\Users\Zhenbang/.cache\torch\checkpoints\resnet50-19c8e357.pth
100%|█████████████████████████████████████████████████████████████████████████████| 97.8M/97.8M [00:22<00:00, 4.60MB/s]


In [9]:
# Training method
def train(net, optim, train_loader):
    net.train()
    for image_cpu, label_cpu in tqdm(train_loader):
        # Move image and label to GPU
        image = image_cpu.to(device)
        label = label_cpu.to(device)
        
        # Clear gradient
        optim.zero_grad()
        
        # Forward through the network
        output = net(image)
        
        # Loss and gradient
        loss = torch.nn.functional.cross_entropy(output, label)
        loss.backward()
        
        # Update paramters
        optim.step()

In [10]:
# Evaluation method
def evaluate(net, val_loader):
    total = 0
    correct = 0
    net.eval()
    
    for image_cpu, label_cpu in tqdm(val_loader):
        # Move image and label to GPU
        image = image_cpu.to(device)
        label = label_cpu.to(device)
        
        # Don't track gradients for performance in evaluation
        with torch.no_grad():
            # Get prediction with forward pass
            prediction = net(image).argmax(dim=-1)
            
            # Total number in batch
            total += image.size(0)
            
            # Number correct in batch
            correct += (prediction == label).sum().item()
            
    return correct/total

### Training

In [11]:
# Create optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

# Create scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer=optimizer, step_size=4, gamma=0.1)

In [15]:
# Start training
num_epochs = 4

print("start")

best_state_dict = {}
best_val_acc = 0
for epoch in range(num_epochs):
    val_acc = evaluate(model, art_val_loader) * 100
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_state_dict = copy.deepcopy(model.state_dict())
        
    print("Epoch {}: {}%".format(epoch, val_acc))
    train(model, optimizer, art_train_loader)
    scheduler.step()
    
    if epoch % 4 == 0 and epoch != 0:
        torch.save(best_state_dict, "./models/art_dept6_date_lr01" + str(epoch))
    
print("Done! {}%".format(evaluate(model, art_val_loader) * 100))

start



Epoch 0: 0.0%



Epoch 1: 49.0%



Epoch 2: 0.6%


OSError: Caught OSError in DataLoader worker process 2.
Original Traceback (most recent call last):
  File "C:\Users\Zhenbang\Anaconda3\lib\site-packages\torch\utils\data\_utils\worker.py", line 178, in _worker_loop
    data = fetcher.fetch(index)
  File "C:\Users\Zhenbang\Anaconda3\lib\site-packages\torch\utils\data\_utils\fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "C:\Users\Zhenbang\Anaconda3\lib\site-packages\torch\utils\data\_utils\fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "C:\Users\Zhenbang\Anaconda3\lib\site-packages\torchvision\datasets\folder.py", line 138, in __getitem__
    sample = self.loader(path)
  File "C:\Users\Zhenbang\Anaconda3\lib\site-packages\torchvision\datasets\folder.py", line 174, in default_loader
    return pil_loader(path)
  File "C:\Users\Zhenbang\Anaconda3\lib\site-packages\torchvision\datasets\folder.py", line 156, in pil_loader
    im

In [ ]:
# Save model
torch.save(best_state_dict, "./models/letter_shifts_class732_lr01_epoch12")

### Miscellaneous

In [ ]:
# Get testing data from directory
letters_val = torchvision.datasets.ImageFolder(root="./data/text_val",
                                               transform=transform)


letters_val_loader = torch.utils.data.DataLoader(dataset=letters_val,
                                                 batch_size=512,
                                                 shuffle=False,
                                                 num_workers=4,
                                                 pin_memory=True)

# Initialize model
model = torchvision.models.resnet18(pretrained=False)
# model = torchvision.models.resnet50(pretrained=False)

# Set number of output classes
model.conv1 = nn.Conv2d(in_channels=3,
                        out_channels=64,
                        kernel_size=(7,7),
                        stride=(2,2),
                        padding=(3,3),
                        bias=False)

in_features = model.fc.in_features
out_features = 26
model.fc = nn.Linear(in_features, out_features)

model = model.to(device)
model.load_state_dict(torch.load("./models/letter_model_lr01_gamma015_e12"))

val_acc = evaluate(model, letters_val_loader) * 100
print("Done!", val_acc)